In [15]:
import numpy as np
from scipy.integrate import odeint 
import matplotlib.pyplot as plt #for plotting
import sympy as sym
from numpy.linalg import eig

In [1]:
def LV(var, t, parameters):    
    x,y = var 
    a,b,c,d = parameters
    #x = var[0], y = var[1]
    #define the vector field
    xdot = a*x  - b*x*y  #dx/dt
    ydot = -c*y + d*x*y    
          
    return xdot, ydot

def LV_log(var, t, parameters):    
    x,y = var 
    a,b,c,d,k = parameters
    #x = var[0], y = var[1]
    #define the vector field
    xdot = a*(1-x/k)*x - b*x*y  #dx/dt   #preys
    ydot = -c*y + d*x*y #predators
    
    #if k>c/d == True:
        #print("Model has 3 real steady states")
        
    return xdot, ydot
# if k goes to infinity, you get malthusian LV model



# this function does the numerical integration for a given model, given initial conditions and given parameters
# this function returns the solution as 2 x d list, where d is the number of initial conditions.
# it works both for one or many i.c.

def integrator(model, initial_conditions, *parameters, fin_time = 25, nt = 1000):
    
    time_line = np.linspace(0, fin_time, nt)
    
    if type(initial_conditions[0])==int:
        
        sol = odeint(model, initial_conditions, time_line, parameters)
    
    else:
        
        sol = []    
        for ic in initial_conditions:
            sol.append(odeint(model, ic, time_line, parameters))

    
        
    return sol 



#this function uses symbolic calculus to find steady_states
def steady_states_computation(model,parameters, print_ask = False):
    x = sym.Symbol('x')
    y = sym.Symbol('y')
    t = 1
    xdot = model([x,y],t,parameters)[0]
    ydot = model([x,y],t,parameters)[1]
    eq = sym.solve((xdot,ydot),(x,y))
    
    if print_ask:
        print("The steady states are: ")
        for e in eq:
            if e[0]>=0 and e[1]>=0:
                print(f"{e}")
        
    return eq


def stability_check(model, parameters, st_state):
    x = sym.Symbol('x')
    y = sym.Symbol('y')
    t = 1
    xdot = model([x,y],t,parameters)[0]
    ydot = model([x,y],t,parameters)[1]
    a_11 = (sym.diff(xdot, x)).subs([(x,st_state[0]), (y, st_state[1])])
    a_12 = sym.diff(xdot, y).subs([(x,st_state[0]), (y, st_state[1])])
    a_21 = sym.diff(ydot, x).subs([(x,st_state[0]), (y, st_state[1])])
    a_22 = sym.diff(ydot, y).subs([(x,st_state[0]), (y, st_state[1])])
    jac = [[float(a_11),float(a_12)], [float(a_21),float(a_22)]]
    det = np.linalg.det(jac)
    trace = a_11 + a_22
    if det> 1/4*trace**2:
        if trace > 0:
            print ("Unstable Focus")
        elif trace < 0:
            print ("Stable Focus")
        elif trace == 0:
            print ("Center")
    else:
        if det<0:
            print ("Saddle Point")
        elif det>0 and trace<0:
            print ("Stable Node")
        elif det>0 and trace>0:
            print ("Unstable Node")
    return eig(jac)[0]

#this function prints the phase space
def phase_space_plot(solutions, steady_states, labelx, labely, steady_ask = True, marksize = 1, model_name = " " ):
    if steady_ask:
        for st in steady_states:
            p,q = st
            if p >= 0 and q >= 0:
                plt.title(f"{model_name}")
                plt.plot(p,q,"o",label = f"S.s. = [{p},{q}]")
                plt.xlabel(labelx)
                plt.ylabel(labely)
       
        
        
    if type(solutions)==np.ndarray:
        x = solutions[:,0]
        y = solutions[:,1]
        plt.plot(x,y,label = f"i.c. = [{x[0]},{y[0]}]")
        plt.plot(x[0],y[0], marker = "x", markersize = marksize)
        plt.legend(loc="center left", bbox_to_anchor=(1, 1))
        plt.show()
      
    else:
        for i in range(len(solutions)):
            x = solutions[i][:,0]
            y = solutions[i][:,1]
            plt.plot(x,y,label = f"i.c. = [{x[0]},{y[0]}]")
            plt.plot(x[0],y[0], marker = "x", markersize = marksize)
            plt.legend(loc="center left", bbox_to_anchor=(1, 1))
        plt.show()
            
            